## Dataset description

In [ ]:
# Dataset coding format
"""
    FO: previus word is misspelled
    D: drawing 
    IN: insertion of words into text
    MD: modification of a word (correction of a word)
    DL: elimination of a word
    NS: when two words were written together
    SB: sillabification
"""

# Classes:
"""
    Extraversion
    Agreeableness
    Conscientiousness
    Emotional stability (Neuroticism)
    Openness
"""

## Imports

In [25]:
import re, os
import pandas as pd, numpy as np

## Preprocesamiento

In [126]:
def procesar_texto(texto):
    # Eliminar saltos de línea
    texto_sin_saltos = texto.replace('\n', ' ')
    
    # Diccionario de reemplazos de etiquetas
    etiquetas_a_palabras = {
        r"<FO:.*?>": "MISSPELLED",
        "<D>": "DRAW",
        "<IN>": "INSERTION",
        "<MD>": "MODIFICATION",
        "<DL>": "ELIMINATION",
        "<NS>": "TOGETHER",
        "<SB>": "SYLLABIFICATION"
    }
    
    # Función para corregir palabras incorrectas
    def corregir_palabras(texto):
        patron = re.compile(r'(\b\w+\b)\s*<FO:(\w+)>')

        def reemplazar_palabra(match):
            palabra_correcta = match.group(2)
            return palabra_correcta
        
        return patron.sub(reemplazar_palabra, texto)
    
    # Aplicar correcciones de palabras al texto sin etiquetas y con etiquetas corregidas
    texto_corregido = corregir_palabras(texto_sin_saltos)
    
    # Reemplaza las etiquetas según el diccionario en el texto original
    for etiqueta, palabra in etiquetas_a_palabras.items():
        texto_sin_saltos = re.sub(etiqueta, palabra, texto_sin_saltos, flags=re.IGNORECASE)
        texto_corregido = re.sub(etiqueta, '', texto_corregido, flags=re.IGNORECASE)
    
    # Texto con etiquetas corregidas
    texto_con_etiquetas = texto_sin_saltos
    
    # Eliminar todas las etiquetas restantes para obtener el texto sin etiquetas
    texto_sin_etiquetas = re.sub(r" ?(MISSPELLED|DRAW|INSERTION|MODIFICATION|ELIMINATION|TOGETHER|SYLLABIFICATION) ?", " ", texto_sin_saltos)
    
    # Corregir palabras en el texto con etiquetas corregidas
    texto_corregido_con_etiquetas = corregir_palabras(texto_con_etiquetas)
    
    # Función para separar signos de puntuación
    def separar_signos(texto):
        signos_puntuacion = ['.', ',', ';', ':', '!', '?', "\"", "\t"]
        for signo in signos_puntuacion:
            texto = texto.replace(signo, f" {signo} ")
        return texto
    
    # Aplicar separación de signos de puntuación a todas las versiones del texto
    texto_sin_etiquetas = separar_signos(texto_sin_etiquetas)
    texto_con_etiquetas = separar_signos(texto_con_etiquetas)
    texto_corregido = separar_signos(texto_corregido)
    texto_corregido_con_etiquetas = separar_signos(texto_corregido_con_etiquetas)

    respuestas = [texto_sin_etiquetas, texto_con_etiquetas, texto_corregido, texto_corregido_con_etiquetas]
    for i in range(4):
        respuestas[i] = re.sub(' +', ' ', respuestas[i]) 
        # nececito borrar los tabuladores definidos como \t
        respuestas[i] = respuestas[i].replace('\t ', '')
    
    return texto_con_etiquetas, texto_sin_etiquetas, texto_corregido_con_etiquetas, texto_corregido

## Probando prepocesamiento

In [127]:
texto = """Mi <MD> vida ha tenido el evento mas <FO:más>
relevante , al entrar a la universidad <DL> ; E so <FO:Eso>  ha sido motivo de alegria <FO:alegría> y motivacion <FO:motivación> isara <FO:para> mi familia."""

texto_sin_etiquetas, texto_con_etiquetas, texto_corregido, texto_corregido_con_etiquetas = procesar_texto(texto)

print("\nTexto con etiquetas corregidas:")
print(texto_con_etiquetas)
print("\nTexto sin etiquetas:")
print(texto_sin_etiquetas)
print("\nTexto con palabra corregida (con etiquetas corregidas):")
print(texto_corregido_con_etiquetas)
print("\nTexto con palabra corregida (sin etiquetas):")
print(texto_corregido)


Texto con etiquetas corregidas:
Mi vida ha tenido el evento mas relevante  ,  al entrar a la universidad  ;  E so  ha sido motivo de alegria y motivacion isara mi familia . 

Texto sin etiquetas:
Mi MODIFICATION vida ha tenido el evento mas MISSPELLED relevante  ,  al entrar a la universidad ELIMINATION  ;  E so MISSPELLED  ha sido motivo de alegria MISSPELLED y motivacion MISSPELLED isara MISSPELLED mi familia . 

Texto con palabra corregida (con etiquetas corregidas):
Mi  vida ha tenido el evento más relevante  ,  al entrar a la universidad   ;  E Eso  ha sido motivo de alegría y motivación para mi familia . 

Texto con palabra corregida (sin etiquetas):
Mi MODIFICATION vida ha tenido el evento mas MISSPELLED relevante  ,  al entrar a la universidad ELIMINATION  ;  E so MISSPELLED  ha sido motivo de alegria MISSPELLED y motivacion MISSPELLED isara MISSPELLED mi familia . 


## Data

In [128]:
# Lee el archivo HWxPIs_train.data y asigna nombres de columna
ids_train = pd.read_csv('../Data/hwxpi/trainval_data/train/HWxPIs_train.data', 
                    sep='\t', 
                    header=None, 
                    names=['ID', 'Transcript', 'Image'])
ids_dev = pd.read_csv('../Data/hwxpi/trainval_data/validation/HWxPIs_valid.data', 
                    sep='\t', 
                    header=None,
                    names=['ID', 'Transcript', 'Image'])
ids_test = pd.read_csv('../Data/hwxpi/test/HWxPIs_test.data', 
                    sep='\t', 
                    header=None,
                    names=['ID', 'Transcript', 'Image'])

In [135]:
for i in ids_train.iterrows():
    print(i[1].ID)

DSC_0085
DSC_0084
DSC_0087
DSC_0086
DSC_0081
DSC_0080
DSC_0083
DSC_0082
DSC_0089
DSC_0088
DSC_0243
DSC_0242
DSC_0241
DSC_0240
DSC_0247
DSC_0246
DSC_0245
DSC_0244
DSC_0249
DSC_0248
DSC_0104
DSC_0105
DSC_0069
DSC_0068
DSC_0100
DSC_0101
DSC_0102
DSC_0103
DSC_0063
DSC_0062
DSC_0061
DSC_0060
DSC_0067
DSC_0066
DSC_0065
DSC_0064
ANR_0044
ANR_0045
ANR_0046
ANR_0047
ANR_0040
ANR_0041
ANR_0042
ANR_0043
ANR_0123
ANR_0122
ANR_0121
ANR_0120
ANR_0048
ANR_0049
ANR_0125
ANR_0124
ANR_0118
ANR_0119
ANR_0110
ANR_0111
ANR_0092
DSC_0209
DSC_0208
DSC_0207
DSC_0206
DSC_0205
DSC_0204
DSC_0203
DSC_0202
DSC_0201
DSC_0200
DSC_0027
DSC_0026
DSC_0025
DSC_0024
DSC_0023
DSC_0022
DSC_0021
DSC_0020
ANR_0094
DSC_0029
DSC_0028
DSC_0159
DSC_0158
DSC_0153
DSC_0152
DSC_0151
DSC_0150
DSC_0157
DSC_0156
DSC_0155
DSC_0154
DSC_0107
ANR_0019
ANR_0018
DSC_0106
ANR_0013
ANR_0012
ANR_0011
ANR_0010
ANR_0017
ANR_0016
ANR_0015
ANR_0014
ANR_0080
ANR_0081
ANR_0082
ANR_0083
ANR_0084
ANR_0085
ANR_0086
ANR_0087
ANR_0088
ANR_0089
DSC_0276
D

In [136]:
# Leyendo y guardando los nombres de los archivos de las transcripciones del dataset
transcripts_train, transcripts_dev, transcripts_test = [], [], []

# Agrega el path a cada archivo para que sea accesible desde la carpeta principal
for i in ids_train.iterrows():
    transcripts_train.append(f'../Data/hwxpi/trainval_data/train/transcripts/{i[1].ID}_hxpi.txt')

for i in ids_dev.iterrows():
    transcripts_dev.append(f'../Data/hwxpi/trainval_data/validation/transcripts/{i[1].ID}_hxpi.txt')

for i in ids_test.iterrows():
    transcripts_test.append(f'../Data/hwxpi/test/transcripts/{i[1].ID}_hxpi.txt')


transcripts = [transcripts_train, transcripts_dev, transcripts_test]

In [138]:
len(transcripts_train), len(transcripts_dev), len(transcripts_test)

(418, 125, 293)

In [140]:
columns=['raw_text', 'raw_text_no_lbls', 'corrected_text', 'corrected_text_no_lbls']
hwxpis_train_data, hwxpis_dev_data, hwxpis_test_data = [], [], []

for i, dataset in enumerate(transcripts):
    for f in dataset:
        with open(f, 'r') as file:
            texto = file.read()
            texto_con_etiquetas, texto_sin_etiquetas, texto_corregido_con_etiquetas, texto_corregido = procesar_texto(texto)
            data_dict = {'raw_text': texto_con_etiquetas, 
                         'raw_text_no_lbls': texto_sin_etiquetas, 
                         'corrected_text': texto_corregido_con_etiquetas, 
                         'corrected_text_no_lbls': texto_corregido}
            if i == 0:
                hwxpis_train_data.append(data_dict)
            elif i == 1:
                hwxpis_dev_data.append(data_dict)
            else:
                hwxpis_test_data.append(data_dict)

hwxpis_train = pd.DataFrame(hwxpis_train_data, columns=columns)
hwxpis_dev = pd.DataFrame(hwxpis_dev_data, columns=columns)
hwxpis_test = pd.DataFrame(hwxpis_test_data, columns=columns)


In [ ]:
# Agrega una columna con el ID de cada transcripción al principio de cada dataframe


In [141]:
hwxpis_train.head()

,raw_text,raw_text_no_lbls,corrected_text,corrected_text_no_lbls
0,Fuí MISSPELLED con mí MISSPELLED familia a Ve...,"Fuí con mí familia a Veracruz , me la pase m...",Fuí MISSPELLED con mí MISSPELLED familia a Ve...,"Fui con mi familia a Veracruz , me la pase m..."
1,-Mi entrada a la Universidad -Creación de emp...,-Mi entrada a la Universidad -Creación de emp...,-Mi entrada a la Universidad -Creación de emp...,-Mi entrada a la Universidad -Creación de emp...
2,Nací en Guanajuato pero me mudé a México cuand...,Nací en Guanajuato pero me mudé a México cuand...,Nací en Guanajuato pero me mudé a México cuand...,Nací en Guanajuato pero me mudé a México cuand...
3,Entrar a la universidad y crear mi propia empr...,Entrar a la universidad y crear mi propia empr...,Entrar a la universidad y crear mi propia empr...,Entrar a la universidad y crear mi propia empr...
4,Paso por un momento confuso mi familia me apo...,Paso por un momento confuso mi familia me apo...,Paso por un momento confuso mi familia me apo...,Paso por un momento confuso mi familia me apo...
